<a href="https://colab.research.google.com/github/PradyothPrakash08/Major-Project/blob/main/Major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Sentiment Analysis of Airbnb reviews

In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 7.1MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/ML /reviews.csv')
df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...
...,...,...,...,...,...,...
84844,3624990,50436321,2015-10-12,37419458,Ryan,The description and pictures of the apartment ...
84845,3624990,51024875,2015-10-17,6933252,Linda,We had an excellent stay. It was clean and com...
84846,3624990,51511988,2015-10-20,19543701,Jaime,"Gran ubicación, cerca de todo lo atractivo del..."
84847,3624990,52814482,2015-11-02,24445024,Jørgen,"Very good apartement, clean and well sized. Si..."


In [4]:
reviews = df[['id','comments']]
reviews = reviews.applymap(str)
reviews

,id,comments
0,38917982,Cute and cozy place. Perfect location to every...
1,39087409,Kelly has a great room in a very central locat...
2,39820030,"Very spacious apartment, and in a great neighb..."
3,40813543,Close to Seattle Center and all it has to offe...
4,41986501,Kelly was a great host and very accommodating ...
...,...,...
84844,50436321,The description and pictures of the apartment ...
84845,51024875,We had an excellent stay. It was clean and com...
84846,51511988,"Gran ubicación, cerca de todo lo atractivo del..."
84847,52814482,"Very good apartement, clean and well sized. Si..."


In [5]:
import nltk
nltk.download('stopwords')
stopword_list =  nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
!pip install contractions
import contractions 
def con(text):
  expand = contractions.fix(text)
  return expand

     |████████████████████████████████| 317kB 4.9MB/s 
     |████████████████████████████████| 245kB 33.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81704 sha256=227233b45813af1ba83744dc3a7879afc3843f825a7bc0b44896914b2c478764
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [7]:
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]' 
  text = re.sub(pattern,'',text)
  return text

In [8]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

In [9]:
def remove_stopwards(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [10]:

reviews.comments = reviews.comments.apply(con)

reviews.comments = reviews.comments.apply(remove_sp)

reviews.comments = reviews.comments.apply(lambda x:x.lower())

reviews.comments = reviews.comments.apply(remove_stopwards)

reviews.head()


,id,comments
0,38917982,cute cozy place perfect location everything
1,39087409,kelly great room central location beautiful bu...
2,39820030,spacious apartment great neighborhood kind apa...
3,40813543,close seattle center offer ballet theater muse...
4,41986501,kelly great host accommodating great neighborh...


In [11]:
reviews['compound'] = reviews['comments'].apply(lambda x: vs.polarity_scores(x)['compound'])
reviews.head()

,id,comments,compound
0,38917982,cute cozy place perfect location everything,0.7717
1,39087409,kelly great room central location beautiful bu...,0.9867
2,39820030,spacious apartment great neighborhood kind apa...,0.9325
3,40813543,close seattle center offer ballet theater muse...,0.8313
4,41986501,kelly great host accommodating great neighborh...,0.9769


In [12]:
def sentiment(reviews):
    if reviews['compound'] >= 0.05:
        return 'positive'
    elif reviews['compound'] <= -0.05:
        return 'negative' 
    else:
        return 'neutral'

reviews['sentiment'] = reviews.apply(sentiment, axis=1)
reviews.head()

,id,comments,compound,sentiment
0,38917982,cute cozy place perfect location everything,0.7717,positive
1,39087409,kelly great room central location beautiful bu...,0.9867,positive
2,39820030,spacious apartment great neighborhood kind apa...,0.9325,positive
3,40813543,close seattle center offer ballet theater muse...,0.8313,positive
4,41986501,kelly great host accommodating great neighborh...,0.9769,positive


In [13]:
reviews.to_csv('reviews_sentiment.csv',index=False)

In [14]:
%%writefile app1.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

df = pd.read_csv('/content/reviews_sentiment.csv')
df = pd.read_csv('/content/reviews_sentiment.csv')
df['comments'] = df['comments'].astype('unicode')
df['sentiment'] = df['sentiment'].astype('unicode')
x = df.iloc[:,1].values # comments column as input
y = df.iloc[:,3].values # sentiment column as output
st.title("Sentiment Analysis of Airbnb Reviews")
st.subheader('TFIFD Vectorizer')

text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',MultinomialNB())])
text_model.fit(x,y)
message = st.text_area("Enter Text","Type Here ..")
op = text_model.predict([message])
if st.button("Predict"):
  st.title(op)

Writing app1.py


In [15]:
!pip install streamlit

     |████████████████████████████████| 7.4MB 6.1MB/s 
     |████████████████████████████████| 133kB 36.4MB/s 
     |████████████████████████████████| 6.9MB 48.1MB/s 
     |████████████████████████████████| 163kB 49.0MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 4.5MB 45.5MB/s 
     |████████████████████████████████| 112kB 51.6MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 143kB 46.8MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 122kB 44.3MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74841 sha256=e3318d9c27dc149d2fb5a8afb15c7681e00b9df8311cae598cf5145014d3acfd
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=939393971289ddde6b2d2abe41ed353c2f0ecaa9

In [16]:
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp36-none-any.whl size=15971 sha256=8fa1c7246b4cae180cfa361ef8d54fde222773d29f7d11425e958851e57d7c09
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [17]:
from pyngrok import ngrok
url = ngrok.connect(port='8501')
url

'http://f86c04ed42f1.ngrok.io'

In [18]:
!streamlit run app1.py &

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.70.217.92:8501

  Stopping...
